# Interface of Neural Force Field to ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through the last tutorial, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from torch.optim import Adam

from nff.io import NeuralFF
from nff.data import Dataset, GraphLoader
from nff.train import load_model, evaluate
import nff.utils.constants as const

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [2]:
dataset = Dataset.from_file('dataset.pth.tar')
loader = GraphLoader(dataset, batch_size=50, cutoff=5, device=0)

Finished Batching 20 Batches


### Creating Atoms

As an example, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [3]:
nxyz, energy, force, _ = dataset[0]
atoms = Atoms(positions=nxyz[:, 1:], numbers=nxyz[:, 0])

## Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [4]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

In [5]:
model = load_model('sandbox/')

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

## Calculating with ASE

Now, we just have to calculate the energies of interest, such as the potential energy of the system. As ASE uses eV as units of energy, we have to convert that information from our database. Calculating is as easy as doing the following:

In [7]:
print('result: %.5f' % atoms.get_potential_energy())
print('  true: %.5f' % (energy / const.EV_TO_KCAL_MOL))

AttributeError: 'NoneType' object has no attribute 'to'

In [ ]:
print(atoms.get_forces())
print((force / const.EV_TO_KCAL_MOL))